In [ ]:
import pandas as pd
from sqlalchemy import engine
import numpy as np
import matplotlib.pyplot
%matplotlib inline

In [ ]:
eng = engine.create_engine(rs_url, connect_args={'connect_timeout': 24 * 60 * 60, 'sslmode': 'allow'})

In [ ]:
tkt_df = pd.read_sql("""
                             with games as(
                                     	  select distinct event_cod
                                      	    from dw.ticket_fac tf 
                                      inner join dw.event_dim ed on tf.event_id = ed.event_id 
                                      inner join dw.date_dim edt on tf.event_date_id = edt.date_id 
                                      inner join dw.date_dim sdt on tf.sale_date_id = sdt.date_id
                                     group by ed.event_cod 		  
                                     having count(*) > 3000
                                     	 and count(distinct date_part(day,sdt.full_date)) > 20
                                    	order by count(*) )
                              select ed.event_cod,
                                        case substring(ed.event_cod,3,1) 
                                            when 'W' THEN 'WNT'
                                            when 'M' THEN 'MNT'
                                        end as team,
                             	  	 sum(tf.ticket_cnt) as tkt_sold,
                             	  	 cast(date_part(day,sdt.full_date) as int) as sold_days_b4_game
                              	    from dw.ticket_fac tf 
                              inner join dw.event_dim ed on tf.event_id = ed.event_id 
                              inner join dw.date_dim edt on tf.event_date_id = edt.date_id 
                              inner join dw.date_dim sdt on tf.sale_date_id = sdt.date_id
                              inner join games on games.event_cod = ed.event_cod
                             group by ed.event_cod,
                             			date_part(day,sdt.full_date),
                                          case substring(ed.event_cod,3,1) 
                                            when 'W' THEN 'WNT'
                                            when 'M' THEN 'MNT'
                                        end
                                        order by event_cod, date_part(day,sdt.full_date) desc"""
                    , con=eng.connect())


In [ ]:
tkt_df = pd.concat([tkt_df, pd.get_dummies(tkt_df.team)], axis=1).drop("team", axis=1).sort_index()

In [ ]:
all_tkt_df = tkt_df.loc[~(tkt_df.event_cod.isin(["19W1110", "15W0920", "18M0907", "19M0605", "18W1010", "19M1011"]) ) & ~(tkt_df['event_cod'].str.contains("C"))].reset_index(drop=True).sort_values(["event_cod","sold_days_b4_game"], ascending=False).sort_index()

In [ ]:
tkt_sub_df = tkt_df[tkt_df.event_cod.isin(["19M0605"])].sort_values("sold_days_b4_game", ascending=False)

In [ ]:
tkt_sub_df

In [ ]:
%matplotlib inline
plt = all_tkt_df.pivot(columns='event_cod', values='tkt_sold').plot()
plt.invert_xaxis()

In [ ]:
## NOT NEEDED TO RUN, THE TRAINING IS THE OLD GAMES

test= tkt_sub_df.reset_index(drop=True)
train = all_tkt_df.reset_index(drop=True)


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import Pipeline

X_scaler = StandardScaler()
y_scaler = MinMaxScaler()
scaler_out = MinMaxScaler()

oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

b = y_scaler.fit_transform(train['tkt_sold'].to_numpy().reshape(-1,1))
c = X_scaler.fit_transform(train['sold_days_b4_game'].to_numpy().reshape(-1,1))

train_enc = pd.concat((
                         pd.DataFrame(b)
                        ,pd.DataFrame(c)
                        ,train.iloc[:,-2:])
                        , axis=1).to_numpy()

train_enc

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
length=7
batch_size=25
features=4
model = Sequential()
model.add(LSTM(128
              ,return_sequences=True
              ,activation='relu'
              ,batch_input_shape =(batch_size, length, features)))
#model.add(Dropout(0.1))
model.add(LSTM(312 
             ,return_sequences=True
              ,activation='relu'
              ))
model.add(LSTM(256 
             ,return_sequences=True
              ,activation='relu'
              ))
#model.add(Dropout(0.1))
model.add(LSTM(128
                ,return_sequences=True
                ,activation='relu'))
model.add(LSTM(64 
               ,activation='relu'
                ,return_sequences=False))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor = 'val_loss', patience = 4)

In [ ]:
from tensorflow import keras
keras.backend.clear_session()

In [ ]:
generator = TimeseriesGenerator(train_enc, train_enc[:,0].reshape(-1,1,1), length=12, batch_size=25) 
len(generator)

In [ ]:
#the length of the generator is 85, batch is 25, what is the length
'''
generator = 85
batch = 25
total_length_of_data = 2132
then length =7

length = total_length_of_data -(generator * batch)
'''

In [ ]:
generator = TimeseriesGenerator(train_enc, train_enc[:,0].reshape(-1,1,1), length=length, batch_size=batch_size) 
    
    
model.fit(generator
      ,epochs=15000
    )


In [ ]:
loss = pd.DataFrame(model.history.history)
loss.plot(figsize=(15,5))

In [ ]:
old_weights= model.get_weights()

length=7
batch_size=1
features=4
n_model = Sequential()
n_model.add(LSTM(128
              ,return_sequences=True
              ,activation='relu'
              ,batch_input_shape =(batch_size, length, features)))
#model.add(Dropout(0.1))
n_model.add(LSTM(312 
             ,return_sequences=True
              ,activation='relu'
              ))
n_model.add(LSTM(256 
             ,return_sequences=True
              ,activation='relu'
              ))
#model.add(Dropout(0.1))
n_model.add(LSTM(128
                ,return_sequences=True
                ,activation='relu'))
n_model.add(LSTM(64 
               ,activation='relu'
                ,return_sequences=False))
n_model.add(Dense(1))

n_model.set_weights(old_weights)
n_model.compile(optimizer='adam', loss='mse')

In [ ]:
tst=tkt_df.loc[tkt_df.event_cod == '19W1110'].sort_values("sold_days_b4_game", ascending=False)

b1 = y_scaler.transform(tst['tkt_sold'].to_numpy().reshape(-1,1))
c1 = X_scaler.transform(tst['sold_days_b4_game'].to_numpy().reshape(-1,1))


tst_enc = pd.concat([
                        pd.DataFrame(b1), 
                        pd.DataFrame(c1),
                        tst.iloc[:,-2:].reset_index(drop=True)],
                            axis=1).to_numpy()


x = tst_enc[-(length+15):-15]

tst_enc

pred = n_model.predict(x.reshape(-1,length,features))


In [ ]:
tst.iloc[-(length+15):-14]

In [ ]:
y_scaler.inverse_transform(pred)

In [ ]:
y_scaler.inverse_transform(x[:,0].reshape(-1,1))

In [ ]:
y_scaler.transform((tst.iloc[-2][1]).reshape(-1,1))

In [ ]:
tst_enc[-15:-8].reshape((-1, length, 4))

In [ ]:
test_predictions = []
current_batch = None

first_eval_batch = tst_enc[-22:(-22+length)]
current_batch = first_eval_batch.reshape((1, length, features))

for i in range(15):
    z=16+i
    
    # get prediction 1 time stamp ahead ([0] is for grabbing just the number instead of [array])
    current_pred = n_model.predict(current_batch)[0]
    
    # store prediction
    test_predictions.append([current_pred[0], tst_enc[z][1], tst_enc[z][2], tst_enc[z][3]])
    
    # update batch to now include prediction and drop first value
    current_batch = np.append(current_batch[:,1:,:], np.asarray([current_pred[0], tst_enc[z][1], tst_enc[z][2], tst_enc[z][3]]).reshape(1,1,features), axis=1)

In [ ]:
tst_enc

In [ ]:
pred_df =pd.concat(
        [pd.DataFrame(y_scaler.inverse_transform(np.asarray(test_predictions)[:,0].reshape(-1,1))),
        pd.DataFrame(X_scaler.inverse_transform(np.asarray(test_predictions)[:,1].reshape(-1,1)))]
    ,axis=1
)
pred_df.columns = ["sales", "days_left"]
pred_df

In [ ]:
pred_df.plot(x="days_left").invert_xaxis()

tst.loc[tst.sold_days_b4_game < 16].drop(["MNT","WNT"], axis=1).rename(columns={"sold_days_b4_game":"days_left"}).plot(x="days_left").invert_xaxis()